# Demo: FAIR data with Multi-Order Coverage maps

## MOCs:

- IVOA	standard to	describe arbitrary sky regions : https://www.ivoa.net/documents/MOC/
- Based on HEALPix sky tessellation	

    - footprints of surveys
    - arbitrary polygons from a list of sky coordinates
    - sky coverage of catalogues

- Perform fast logical operations (intersection, union, difference, complement)

- Space and Time coverages


<font color="blue">Check the MOCPy's documentation @ https://cds-astro.github.io/mocpy/ </font>

In [ ]:
from pathlib import Path

# Find and access data
from astroquery.cds import cds
from astroquery.simbad import Simbad
import pyvo

# Work with MOCs
from mocpy import STMOC
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
from astropy import units as u

# Visualize
from mocpy import World2ScreenMPL
import ipyaladin.aladin_widget as ipyal
import matplotlib.pyplot as plt

## Use `astroquery.cds` to get spatial footprints (MOCs)
Query the CDS MOCServer <http://alasky.unistra.fr/MocServer/query> to retrieve the data-sets based on their
meta data values

In [ ]:
info_1 = cds.find_datasets(meta_data="ID=*SDSS*")
print(info_1['ID']) 

info_2 = cds.find_datasets(meta_data="ID=*XMM*")
print(info_2['ID']) 

#### Get the MOC object corresponding to the <ins>union</ins> of the MOCs from all the retrieved data-sets

In [ ]:
moc_sdss = cds.find_datasets(meta_data="ID=CDS/P/SDSS9/color*", return_moc=True)
moc_xmm = cds.find_datasets(meta_data="ID=ESAVO/P/XMM/EPIC-RGB", return_moc=True)

#### Is a given source observed by a specific survey? 

In [ ]:
m87 = Simbad.query_object('M87')
m87

In [ ]:
ra, dec = Angle('12h30m49.4233s'), Angle('12d23m28.043s')

print(moc_xmm.contains(ra, dec))

## Compute the intersection of the MOCs
What fraction of the sky is covered by both SDSS and XMM ?

In [ ]:
moc_intersection = moc_sdss.intersection(moc_xmm)

sky_frac = moc_intersection.sky_fraction
print(f'The intersection of SDSS and XMM covers {round(sky_frac * 100., 1)} % of the sky')

## Save into a fits file

In [ ]:
moc_intersection.serialize(format='fits')
moc_intersection.write("Data" / Path("moc_intersection.fits"), format="fits", overwrite=True)

## Visualize and interact with MOCs in Aladin Lite using `ipyaladin`

In [ ]:
aladin = ipyal.Aladin(target='M87', survey='P/SDSS9/color')
aladin

In [ ]:
aladin.add_moc_from_dict(moc_sdss.serialize(format='json'), {'color': 'yellow', 'opacity': 0.3, 'name': 'SDSS'})
aladin.add_moc_from_dict(moc_xmm.serialize(format='json'), {'color': 'blue', 'opacity': 0.5, 'name': 'XMM'})
aladin.add_moc_from_dict(moc_intersection.serialize(format='json'), {'color': 'magenta', 'opacity': 0.5, 'name': 'intersection'})

## Filter a table by a MOC

I have a catalogue of sources, how I can select those observed in SDSS and with XMM?

In [ ]:
tap_vizier = pyvo.dal.TAPService('https://tapvizier.cds.unistra.fr/TAPVizieR/tap')

query = """SELECT  *  FROM "VII/192/arplist" """

table_arplist = tap_vizier.search(query).to_table()
table_arplist.show_in_notebook(display_length=5)

In [ ]:
coords = SkyCoord(ra=table_arplist['RAJ2000'], 
                  dec=table_arplist['DEJ2000'], 
                  unit=(u.deg, u.deg))

indexes = moc_intersection.contains(coords.ra, 
                                    coords.dec)

print(f'{len(table_arplist[indexes])} peculiar galaxies are observed both by SDSS and XMM')

In [ ]:
aladin.add_table(table_arplist[indexes]['Name', 'RAJ2000', 'DEJ2000'])

## 2. Space & Time coverage: STMOC

In [ ]:
# Find catalog on Vizier 
tap_vizier = pyvo.dal.TAPService('https://tapvizier.cds.unistra.fr/TAPVizieR/tap')

query = '''
SELECT  *  FROM tables 
WHERE description LIKE '%SDSS%quasar%catalog%'
'''

sdss_catalogues = tap_vizier.search(query).to_table()
sdss_catalogues

In [ ]:
# Access the catalog 
# Check the first rows to explore the table content and columns

query = """SELECT 
TOP 3 *
FROM "VII/286/dr14q" """

table_info = tap_vizier.search(query).to_table()
table_info

In [ ]:
# Access the info on position of the sources and observation time

query = """SELECT 
RAJ2000, DEJ2000, MJD
FROM "VII/286/dr14q" """

table_sdss = tap_vizier.search(query).to_table()
table_sdss

In [ ]:
# Define times, longitudes and latitudes
times_sdss = Time(table_sdss['MJD'].data, format='mjd', scale='tdb')
lon_sdss = table_sdss['RAJ2000'].quantity
lat_sdss = table_sdss['DEJ2000'].quantity

## Create the Space-Time coverage 
at the depth (time, space) = 10, 9 i.e.:

- a **time** resolution of ~3 days
- a **spatial** resolution of ~7 arcmin

In [ ]:
time_depth = 10
spatial_depth = 9

sdss = STMOC.from_times_positions(times_sdss, time_depth, lon_sdss, lat_sdss, spatial_depth)

print("Time of the first observation: ", sdss.min_time.iso)
print("Time of the last observation: ", sdss.max_time.iso)

## Query a ST-MOC by a time range
Retrieve the regions being observed each four years

In [ ]:
def add_to_plot(fig, id, wcs, title, moc):
    ax = fig.add_subplot(id, projection=wcs)

    ax.grid(color="black", linestyle="dotted")
    ax.set_title(title)
    ax.set_xlabel('lon')
    ax.set_ylabel('lat')

    #Draw the MOC with its HEALPix cells
    moc.fill(ax=ax, wcs=wcs, alpha=0.9, fill=True, linewidth=0, color="#00bb00")
    #Draw the perimeter of a MOC
    moc.border(ax=ax, wcs=wcs, linewidth=1, color="green")

fig = plt.figure(figsize=(10, 10))

time_ranges = Time([
        [["2000-03-02", "2004-01-01"]],
        [["2004-01-01", "2008-01-01"]],
        [["2008-01-01", "2012-05-14"]],
        [["2012-01-01", "2016-05-14"]]
    ], format='iso', scale='tdb', out_subfmt="date")

with World2ScreenMPL(fig, 
         fov=330 * u.deg,
         center=SkyCoord(0, 0, unit='deg', frame='galactic'),
         coordsys="galactic",
         rotation=Angle(0, u.degree),
         projection="AIT") as wcs:

    for i in range(0, 4):
        moc_sdss = sdss.query_by_time(time_ranges[i])
        title = "Quasars observations between \n{0} and {1}".format(time_ranges[i][0, 0].iso, time_ranges[i][0, 1].iso)
        id_subplot = int("22" + str(i+1))
        add_to_plot(fig, id_subplot, wcs, title, moc_sdss)

plt.show()

## Compute the intersection between Space-Time coverages

#### Retrieve the areas that have been observed at the same time by two surveys

In [ ]:
query = '''
SELECT  *  FROM tables 
WHERE description LIKE '%3XMM%'
'''

mash_catalogues = tap_vizier.search(query).to_table()
mash_catalogues

In [ ]:
query = """SELECT 
RA_ICRS,DE_ICRS,MJD0
FROM "IX/55/xmm3r8s" """

table_xmm = tap_vizier.search(query).to_table()
table_xmm

In [ ]:
times_xmm = Time(table_xmm['MJD0'].data, format='mjd', scale='tdb')
lon_xmm = table_xmm['RA_ICRS'].quantity
lat_xmm = table_xmm['DE_ICRS'].quantity

# Create the STMOC
xmm_dr8_stmoc = STMOC.from_times_positions(times_xmm, time_depth, lon_xmm, lat_xmm, spatial_depth)


In [ ]:
# Compute their intersection and check that it is not empty
xmm_inter_sdss = xmm_dr8_stmoc.intersection(sdss)
assert(not xmm_inter_sdss.is_empty())

### - Discover all the functionalities on https://cds-astro.github.io/mocpy/ 

### - MOC data structure to plan multi-messenger observations 
- Greco et al., Astronomy and Computing 2021, DOI: [10.1016/j.ascom.2022.100547](https://ui.adsabs.harvard.edu/link_gateway/2022A&C....3900547G/doi:10.1016/j.ascom.2022.100547)
- Notebook: https://github.com/ggreco77/MOC-to-plan-MMA
